In [ ]:
print("Hello")

## Conversational App for Itinerary Planning

In [ ]:
import os
from dotenv import load_dotenv
#from langchain import HuggingFaceHub

from langchain import PromptTemplate, LLMChain

load_dotenv()

#os.environ["HUGGINGFACEHUB_API_TOKEN"]
#retrieving api key
key=os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagePlaceHolder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chains import LLMChain, ConversationChain
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

### Sample bot with no memory

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chains import LLMChain, ConversationChain
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()
messages = [
    SystemMessage(content="You are a helpful assistant that helps the user to plan an optimized itinerary."),
    HumanMessage(content="I'm going to Rome for 2 days, what can I visit?")
]
chat(messages)

In [ ]:
output = chat(messages)
print(output.content)

### Adding Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat, verbose=True, memory=memory
)

conversation.run("Hi there!")

In [ ]:
conversation.run("what is the most iconic place in Rome?")

In [ ]:
conversation.run("What kind of other events?")

In [ ]:
memory.load_memory_variables({})

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagePlaceHolder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory

#LLM
repo_id = 'databricks/dolly-v2-3b'
llm = HuggingFaceHub(
    repo_id = repo_id, model_kwargs={"temperature":0.5, "max_length":1000}
)

#Prompt
prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a helpful assistant that help the user to plan an optimized itinerary."
        ),
        # The`variable_name` here is what must align with memory
        MessagePlaceHolder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_message=True)
conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=False,
    memory=memory
)

In [ ]:
while True:
    query = input('you: ')
    if query == 'q':
        break
    output = conversation({"'question": query})
    print('User ', query)
    print('AI System: ', output['text'])

### Adding non parametric knowledge

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader

import os

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200
)

raw_documents = PyPDFLoader('italy_travel.pdf').load()
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key= 'chat_history',
    return_messages=True
)

llm = ChatOpenAI()
qa_chain = ConversationalRetrievalChain.from_llm(llm,  retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Give me some review about the Pantheon'})